# Specific arguments for particular field geometry

The openPMD format supports 3 types of geometries:
- Cartesian 2D
- Cartesian 3D
- Cylindrical with azimuthal decomposition (thetaMode)

This notebook shows how to use the arguments of `get_field` which are specific to a given geometry.

## (optional) Preparing this notebook to run it locally

If you choose to run this notebook on your local machine, you will need to download the openPMD data files which will then be visualized. To do so, execute the following cell. (Downloading the data may take a few seconds.)

In [ ]:
import os
if os.path.exists('./example-2d.tar.gz') == False:
    # 2D Cartesian
    !wget https://github.com/openPMD/openPMD-example-datasets/raw/draft/example-2d.tar.gz
    !tar zxf example-2d.tar.gz
    
if os.path.exists('./example-3d.tar.gz') == False:
    # 3D Cartesian
    !wget https://github.com/openPMD/openPMD-example-datasets/raw/draft/example-3d.tar.gz
    !tar zxf example-3d.tar.gz
    
if os.path.exists('./example-thetaMode.tar.gz') == False:
    # thetaMode
    !wget https://github.com/openPMD/openPMD-example-datasets/raw/draft/example-thetaMode.tar.gz
    !tar zxf example-thetaMode.tar.gz

In addition, we choose here to incorporate the plots inside the notebook.

In [1]:
%matplotlib inline

## 3D Cartesian geometry

## Cylindrical geometry (with azimuthal decomposition)